In [34]:
import pandas as pd
import math
import numpy as np

In [35]:
def treatEV(coeff):
    rr = 1-math.exp(coeff)
    return rr

def treatnonEV(coeff):
    rr= math.exp(coeff)-1
    return rr

def get_baseline_asthma(df):
    mean = df['AC'].mean()
    weight_mean = (df['AC']*(df['year']-2012)).sum()/(df['year']-2012).sum()
    return weight_mean

def deltanonEV(df, unit):
    mean = df['nonev_sales'].mean()
    weight_mean = (df['nonev_sales']*(df['year']-2012)).sum()/(df['year']-2012).sum()
    return weight_mean * (np.exp(unit)-1)

def deltaEV(df, unit):
    mean = df['ev_sales'].mean()
    weight_mean = (df['ev_sales']*(df['year']-2012)).sum()/(df['year']-2012).sum()
    return weight_mean * (np.exp(unit)-1)

def incrementfactor(evcoef, nonevcoef, df):
    ev=treatEV(evcoef)*get_baseline_asthma(df)
    nonev=treatnonEV(nonevcoef)*get_baseline_asthma(df)
    ms = (ev) / (ev + nonev)
    return ms, ev, nonev

def minMS(evcoef, nonevcoef, df):
    ms, ev, nonev = incrementfactor(evcoef, nonevcoef, df)
    return ms, ev, nonev
    

In [36]:
def read_model(f, pollutant, stat):
    df = pd.read_csv("output_files/model_AC_2013_2019_{}.csv".format(pollutant))
    sdf = pd.read_csv("output_files/{}_{}_{}.csv".format(f, pollutant, stat))
    coef_ev_sales_log = sdf.loc[sdf['term']=="ev_sales_log", "estimate"].item()
    coef_ev_sales_log_5 = sdf.loc[sdf['term']=="ev_sales_log", "CI_lower"].item()
    coef_ev_sales_log_95 = sdf.loc[sdf['term']=="ev_sales_log", "CI_upper"].item()
    
    coef_nonev_sales_log = sdf.loc[sdf['term']=="nonev_sales_log", "estimate"].item()
    coef_nonev_sales_log_5 = sdf.loc[sdf['term']=="nonev_sales_log", "CI_lower"].item()
    coef_nonev_sales_log_95 = sdf.loc[sdf['term']=="nonev_sales_log", "CI_upper"].item()
    
    marketshare, ev, nonev= minMS(coef_ev_sales_log,coef_nonev_sales_log,df)
    marketshare_5, ev_5, nonev_5 = minMS(coef_ev_sales_log_5,coef_nonev_sales_log_5,df)
    marketshare_95, ev_95, nonev_95 = minMS(coef_ev_sales_log_95,coef_nonev_sales_log_95,df)
    a = (marketshare, marketshare_5, marketshare_95)
    b = (ev, ev_5, ev_95)
    c = (nonev, nonev_5, nonev_95)
    return a,b,c

def execute(f, pollutant): 
    print(pollutant)
    a, b, c = read_model(f, pollutant, 'AC')
    marketshare_AC, marketshare_AC_5, marketshare_AC_95 = a
    ev, ev_5, ev_95 = b
    nonev, nonev_5, nonev_95 = c
    
    a,b,c= read_model(f, pollutant, 'AC_5')
    LBmarketshare_AC, LBmarketshare_AC_5, LBmarketshare_AC_95 = a
    LBev, LBev_5, LBev_95 = b
    LBnonev, LBnonev_5, LBnonev_95 = c
    
    
    a,b,c = read_model(f, pollutant, 'AC_95')
    UBmarketshare_AC, UBmarketshare_AC_5, UBmarketshare_AC_95 = a
    UBev, UBev_5, UBev_95 = b
    UBnonev, UBnonev_5, UBnonev_95 = c
    
    a = "{:.2f} ({:.2f}-{:.2f})\n".format(marketshare_AC*100, marketshare_AC_5*100, marketshare_AC_95*100)
    LBa = "{:.2f} ({:.2f}-{:.2f})\n".format(LBmarketshare_AC*100, LBmarketshare_AC_5*100, LBmarketshare_AC_95*100)
    UBa = "{:.2f} ({:.2f}-{:.2f})\n".format(UBmarketshare_AC*100, UBmarketshare_AC_5*100, UBmarketshare_AC_95*100)
    MS = (a, LBa, UBa)
    
    EV = "{} ({}-{})".format(round(ev), round(ev_5), round(ev_95))
    LBEV = "{} ({}-{})".format(round(LBev), round(LBev_5), round(LBev_95))
    UBEV = "{} ({}-{})".format(round(UBev), round(UBev_5), round(UBev_95))
    ev_count = (EV, LBEV, UBEV)
    
    NONEV = "{} ({}-{})".format(round(nonev), round(nonev_5), round(nonev_95))
    LBNONEV = "{} ({}-{})".format(round(LBnonev), round(LBnonev_5), round(LBnonev_95))
    UBNONEV = "{} ({}-{})".format(round(UBnonev), round(UBnonev_5), round(UBnonev_95))
    nonev_count = (NONEV, LBNONEV, UBNONEV)

    return MS, ev_count, nonev_count

def open_string(s):
    rs = []
    s = s.strip().split()
    rs.append(s[0])
    s2 = s[1].replace("(", "").replace(")", "").split("-")
    rs.append(s2[0])
    rs.append(s2[1])
    rs = [float(i) for i in rs]
    return rs

def align(string_val):
    rs= []
    for i in string_val:
        rs.extend(open_string(i))
    return rs
    
def get_index(ev):
    min_ev = min(ev)
    min_index = ev.index(min_ev)
    
    max_ev = max(ev)
    max_index = ev.index(max_ev)
    return min_index, max_index

def get_results(f, pollutant):
    rs = execute(f, pollutant)
    ms = align(rs[0])
    ev = align(rs[1])
    nonev = align(rs[2])
    ev = [int(i) for i in ev]
    nonev = [int(i) for i in nonev]

    min_index, max_index = get_index(ev)
    final_result = {
        "EV Sales": "{} (-{}--{})".format(ev[0], ev[max_index], ev[min_index]),
        "non-EV Sales": "{} ({}-{})".format(nonev[0], nonev[max_index], nonev[min_index]),
        "MS": "{}% ({}%-{}%)".format(ms[0], ms[min_index], ms[max_index]),
    }
    return final_result

In [37]:
get_results("results","no2")

no2


{'EV Sales': '264 (-401--113)',
 'non-EV Sales': '971 (563-1471)',
 'MS': '21.38% (7.14%-41.59%)'}

In [40]:
get_results("results",'pm2.5')

pm2.5


{'EV Sales': '250 (-361--131)',
 'non-EV Sales': '613 (336-950)',
 'MS': '29.0% (12.11%-51.83%)'}

In [39]:
get_results("results", 'pm10')

pm10


{'EV Sales': '551 (-802--277)',
 'non-EV Sales': '1089 (519-1747)',
 'MS': '33.62% (13.68%-60.74%)'}

In [164]:
sdf = pd.read_csv("output_files/results_{}_{}.csv".format("no2", "AC"))
coef_ev_sales_log = sdf.loc[sdf['term']=="ev_sales_log", "2.5 %"].item()
coef_nonev_sales_log = sdf.loc[sdf['term']=="nonev_sales_log", "estimate"].item()

In [157]:
sdf

,Unnamed: 0,effect,group,term,estimate,std.error,statistic,p.value,2.5 %,97.5 %
0,.sig01,fixed,NaN,(Intercept),-11.114891,1.029780,-10.793463,3.695944e-27,0.415478,0.682292
1,.sig02,fixed,NaN,nonev_sales_log,0.473295,0.096909,4.883898,1.040090e-06,0.185927,0.748973
2,(Intercept),fixed,NaN,ev_sales_log,-0.179829,0.052295,-3.438755,5.843957e-04,-13.230068,-9.103846
3,nonev_sales_log,fixed,NaN,year_fixed,-0.476844,0.054080,-8.817351,1.171998e-18,0.285030,0.671096
4,ev_sales_log,ran_pars,state_code:EPA_Region,sd__(Intercept),0.526775,NaN,NaN,NaN,-0.283395,-0.078052
5,year_fixed,ran_pars,EPA_Region,sd__(Intercept),0.396819,NaN,NaN,NaN,-0.583423,-0.371208


In [144]:
319068.50326797384*(math.exp(1)-1)

548249.6111989849

In [109]:
# year fixed is year fixed
execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr -0.02857477567983291 0.0281741310945726
nonEV rr 0.24361877618658734 0.218025497340806
x by baseline asthma -45.82505057458781 390.6887271751571
pm2.5
EV rr 0.006724165847561414 -0.0067468749076403
nonEV rr 0.11825253569690952 0.111767230886407
x by baseline asthma 12.841072431811076 225.82568761001568
pm10
EV rr 0.03193333843762114 -0.0324543288262711
nonEV rr 0.09140726325813309 0.0874679307397258
x by baseline asthma 86.3861396819948 247.27513620884636


('-0.86 (nan-nan)\n', '100.00 (nan-nan)\n', '12.30 (nan-nan)\n')

In [111]:
# year fixed is log(year)
execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr -0.03053657525441622 0.0300796134447898
nonEV rr 0.24107081294266108 0.215974565789273
x by baseline asthma -48.97116677615552 386.60258680354553
pm2.5
EV rr 0.006521852442384346 -0.0065432126447342
nonEV rr 0.11799362213038922 0.111535670003209
x by baseline asthma 12.454716540433527 225.33124295522308
pm10
EV rr 0.03379796187343065 -0.0343823174389165
nonEV rr 0.09392566589141982 0.0897727546084846
x by baseline asthma 91.43032323627028 254.08792473327782


('-0.86 (nan-nan)\n', '100.00 (nan-nan)\n', '11.38 (nan-nan)\n')

In [112]:
# year fixed is log(year_fixed)

execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr 0.16458689884059585 -0.179828944543343
nonEV rr 0.6052751812816011 0.473295194397578
x by baseline asthma 263.94618273794185 970.6730896000639
pm2.5
EV rr 0.13103726957708794 -0.140455042527237
nonEV rr 0.3207602461322443 0.278207514845745
x by baseline asthma 250.24056634718096 612.5526418006867
pm10
EV rr 0.20379244192958657 -0.227895375782737
nonEV rr 0.40234156994262027 0.338143389426201
x by baseline asthma 551.2997768477459 1088.4153289775945


('24.69 (nan-nan)\n', '8.20 (nan-nan)\n', '5.01 (nan-nan)\n')

In [113]:
# year is a factor now
execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr -0.14715459363297123 0.137284609913386
nonEV rr 0.08885671496917125 0.0851282604264799
x by baseline asthma -235.99018837698384 142.49852747672793
pm2.5
EV rr -0.024257885798383727 0.0239683365105277
nonEV rr 0.0692225865169751 0.0669318297480327
x by baseline asthma -46.32504248725726 132.1936828348979
pm10
EV rr -0.06257960946918661 0.0606995455688148
nonEV rr -0.029332530938859902 -0.0297713316913538
x by baseline asthma -169.29050169339445 -79.35042932828884


('-0.39 (nan-nan)\n', '-0.80 (nan-nan)\n', '0.51 (nan-nan)\n')

In [114]:
# epa region is also a factor
execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr -0.14715459363297123 0.137284609913386
nonEV rr 0.08885671496917125 0.0851282604264799
x by baseline asthma -235.99018837698384 142.49852747672793
pm2.5
EV rr -0.024257885798383727 0.0239683365105277
nonEV rr 0.0692225865169751 0.0669318297480327
x by baseline asthma -46.32504248725726 132.1936828348979
pm10
EV rr -0.06257960946918661 0.0606995455688148
nonEV rr -0.029332530938859902 -0.0297713316913538
x by baseline asthma -169.29050169339445 -79.35042932828884


('-0.39 (nan-nan)\n', '-0.80 (nan-nan)\n', '0.51 (nan-nan)\n')

In [115]:
# no year
execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr 0.40746395111167755 -0.523343565818043
nonEV rr 1.185662711682252 0.781919083102319
x by baseline asthma 653.4454155030198 1901.4341297380167
pm2.5
EV rr 0.3420439777612374 -0.418617185393812
nonEV rr 0.7492037630841417 0.559160691871569
x by baseline asthma 653.1979717439161 1430.7469515252794
pm10
EV rr 0.4739003220747784 -0.642264582438641
nonEV rr 1.2083292345583883 0.792236227153422
x by baseline asthma 1281.9962278000967 3268.7749901971965


('12.86 (nan-nan)\n', '6.29 (nan-nan)\n', '9.12 (nan-nan)\n')

In [118]:
# factor year and fac year_sq
execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr -0.14715459363297123 0.137284609913386
nonEV rr 0.08885671496917125 0.0851282604264799
x by baseline asthma -235.99018837698384 142.49852747672793
pm2.5
EV rr -0.024257885798383727 0.0239683365105277
nonEV rr 0.0692225865169751 0.0669318297480327
x by baseline asthma -46.32504248725726 132.1936828348979
pm10
EV rr -0.06257960946918661 0.0606995455688148
nonEV rr -0.029332530938859902 -0.0297713316913538
x by baseline asthma -169.29050169339445 -79.35042932828884


('-0.39 (nan-nan)\n', '-0.80 (nan-nan)\n', '0.51 (nan-nan)\n')

In [119]:
#  factor year and fac year_sq with desnity
execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr -0.026503866952466426 0.0261587245934442
nonEV rr 0.7555936162172128 0.56280704258542
x by baseline asthma -42.50395723582537 1211.7370951551052
pm2.5
EV rr 0.013562001236669508 -0.0136548052023164
nonEV rr 0.3826737609246871 0.324019132538
x by baseline asthma 25.899218453028194 730.7882632861521
pm10
EV rr -0.00011874609548523374 0.0001187390457258
nonEV rr 0.5636369084966111 0.447014459478111
x by baseline asthma -0.32123220725315 1524.7518452362333


('-0.86 (nan-nan)\n', '100.00 (nan-nan)\n', '-0.86 (nan-nan)\n')

In [120]:
# year continous with population
execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr -0.028565659172263835 0.0281652678128883
nonEV rr 0.24366159243660257 0.218059925506251
x by baseline asthma -45.81043053259353 390.75739112004146
pm2.5
EV rr 0.006720635615430459 -0.006743320783261
nonEV rr 0.11824857474731876 0.111763688791505
x by baseline asthma 12.834330782731863 225.81812342410075
pm10
EV rr 0.03190719695672728 -0.0324273253884105
nonEV rr 0.09133263202552522 0.0873995476650423
x by baseline asthma 86.31542168849712 247.07324362886146


('-0.86 (nan-nan)\n', '100.00 (nan-nan)\n', '12.30 (nan-nan)\n')

In [121]:
# year continous and year_sq cont with population
execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr -0.034720224374933384 0.034131075562161
nonEV rr 0.23346847041150154 0.209830095583569
x by baseline asthma -55.68043843176244 374.41079447316514
pm2.5
EV rr 0.016113210898018204 -0.0162444402744693
nonEV rr 0.1327411371096081 0.124640480309922
x by baseline asthma 30.77123808978294 253.49442517447883
pm10
EV rr 0.03496579187549376 -0.0355917294377609
nonEV rr 0.09613047794338803 0.0917862306584477
x by baseline asthma 94.58953961072221 260.05238730476646


('-0.86 (nan-nan)\n', '96.97 (nan-nan)\n', '11.06 (nan-nan)\n')

In [122]:
# log(year)
execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr 0.16458689884059585 -0.179828944543343
nonEV rr 0.6052751812816011 0.473295194397578
x by baseline asthma 263.94618273794185 970.6730896000639
pm2.5
EV rr 0.13103726957708794 -0.140455042527237
nonEV rr 0.3207602461322443 0.278207514845745
x by baseline asthma 250.24056634718096 612.5526418006867
pm10
EV rr 0.20379244192958657 -0.227895375782737
nonEV rr 0.40234156994262027 0.338143389426201
x by baseline asthma 551.2997768477459 1088.4153289775945


('24.69 (nan-nan)\n', '8.20 (nan-nan)\n', '5.01 (nan-nan)\n')

In [123]:
# log(year) and log(year^2)
execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr 0.16458689884059585 -0.179828944543343
nonEV rr 0.6052751812816011 0.473295194397578
x by baseline asthma 263.94618273794185 970.6730896000639
pm2.5
EV rr 0.13103726957708794 -0.140455042527237
nonEV rr 0.3207602461322443 0.278207514845745
x by baseline asthma 250.24056634718096 612.5526418006867
pm10
EV rr 0.20379244192958657 -0.227895375782737
nonEV rr 0.40234156994262027 0.338143389426201
x by baseline asthma 551.2997768477459 1088.4153289775945


('24.69 (nan-nan)\n', '8.20 (nan-nan)\n', '5.01 (nan-nan)\n')

In [124]:
# log(year) and density
execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr 0.22724959071613027 -0.257799168324151
nonEV rr 1.484324834351816 0.910000926194527
x by baseline asthma 364.43764613594846 2380.3952606807093
pm2.5
EV rr 0.17857787354080146 -0.196718140326506
nonEV rr 0.8205248877428799 0.599124859368184
x by baseline asthma 341.0283834221396 1566.9481917123017
pm10
EV rr 0.30834294661520323 -0.368665033802812
nonEV rr 1.4741505461677367 0.905897123439761
x by baseline asthma 834.1300396227252 3987.875406208911


('85.36 (nan-nan)\n', '45.30 (nan-nan)\n', '50.12 (nan-nan)\n')

In [125]:
execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr 0.16458689884059585 -0.179828944543343
nonEV rr 0.6052751812816011 0.473295194397578
x by baseline asthma 263.94618273794185 970.6730896000639
pm2.5
EV rr 0.13103726957708794 -0.140455042527237
nonEV rr 0.3207602461322443 0.278207514845745
x by baseline asthma 250.24056634718096 612.5526418006867
pm10
EV rr 0.20379244192958657 -0.227895375782737
nonEV rr 0.40234156994262027 0.338143389426201
x by baseline asthma 551.2997768477459 1088.4153289775945


('24.69 (nan-nan)\n', '8.20 (nan-nan)\n', '5.01 (nan-nan)\n')

In [134]:

a = [-0.179828944543343, -0.140455042527237, -0.227895375782737]
b = [0.473295194397578, 0.278207514845745, 0.338143389426201]

def compute(x1, x2):
    a = 1 - math.exp(x1)
    b =  math.exp(x2) - 1
    return a*1600, b*1600, a/(b+a)

In [136]:
for i, j in zip(a, b):
    print(compute(i, j))

(263.33903814495335, 968.4402900505618, 0.21378751219240683)
(209.6596313233407, 513.2163938115909, 0.29003539200820194)
(326.0679070873385, 643.7465119081925, 0.3362168067423228)


In [133]:
263.3/(968+263)

0.2138911454102356

In [141]:
## at_risk
execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr 0.16570458659363174 -0.181167726624437
nonEV rr 0.6045324937916319 0.472832433024683
x by baseline asthma 265.7386061810282 969.4820499163134
Inc 3.6482544401391057
pm2.5
EV rr 0.13272312634554229 -0.142397006406309
nonEV rr 0.32119849218690866 0.278539273288697
x by baseline asthma 253.46003019803646 613.3895559188804
Inc 2.420064242238192
pm10
EV rr 0.20489580162758225 -0.229282105781235
nonEV rr 0.40169017874788127 0.337678778990345
x by baseline asthma 554.2845880091824 1086.6531840378707
Inc 1.9604607588689964


('24.13 (nan-nan)\n', '7.97 (nan-nan)\n', '4.93 (nan-nan)\n')

In [140]:
a = [-0.181167726624437, -0.142397006406309, -0.229282105781235]
b = [0.472832433024683, 0.278539273288697, 0.337678778990345]

for i, j in zip(a, b):
    print(compute(i, j))


(265.1273385498108, 967.251990066611, 0.2151345226209419)
(212.35700215286766, 513.9175874990539, 0.292392168442301)
(327.8332826041316, 642.70428599661, 0.33778525758336225)


In [155]:
## pop
execute('no2'),execute('pm2.5'), execute('pm10')

no2
EV rr 0.16458689884059585 -0.179828944543343
nonEV rr 0.6052751812816011 0.473295194397578
baseline MS: 0.21
Inc 0.2137875121924068
EV rr 0.1695670668082615 -0.185808107925643
nonEV rr 0.6399975877404553 0.494694770945059
baseline MS: 0.21
Inc 0.20945463201180006
EV rr 0.16140711744845748 -0.176029931579915
nonEV rr 0.583651738270933 0.459733407016303
baseline MS: 0.22
Inc 0.21663673441289558
pm2.5
EV rr 0.13103726957708794 -0.140455042527237
nonEV rr 0.3207602461322443 0.278207514845745
baseline MS: 0.29
Inc 0.2900353920082019
EV rr 0.13094460198455615 -0.140348406593925
nonEV rr 0.3270241429974763 0.282938948852598
baseline MS: 0.29
Inc 0.28592475669861167
EV rr 0.13024641775612145 -0.139545346253332
nonEV rr 0.3132374522513697 0.2724954260767
baseline MS: 0.29
Inc 0.29368918818608075
pm10
EV rr 0.20379244192958657 -0.227895375782737
nonEV rr 0.40234156994262027 0.338143389426201
baseline MS: 0.34
Inc 0.33621680674232285
EV rr 0.21416938460996415 -0.241014011825134
nonEV rr 0.437

('21.38 (21.66-20.95)\n', '29.00 (29.37-28.59)\n', '33.62 (34.30-32.84)\n')